# Capstone assignment - Neighborhoods in Toronto

Hello, welcome to my notebook. This is written for course assignment of applied data science.

Importing necessary libraries for this assignment.

In [1]:
import pandas as pd
import numpy as np

# For converting address to coordinates
!pip install geocoder
import geocoder

# For geo map visualization
!pip install folium
import folium

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libries are installed')

Libries are installed


## Download data

"**Note:** There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe"

Here I am scrapping the table data of neighborhoods in Toronto based on postal code from wikipedia website using pandas `read_html` function.

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


## Data cleaning 

Next we are ignoring the rows of Borough has not assigned and reseting the index of resulted data frame `df`

In [3]:
df= df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Here we are verifying the whether any postal codes are duplicated using `duplicated`. 

In [4]:
df['Postal code'].duplicated().any()

False

There are no duplicates in the postal code. Next checking the whether there are any NaN values in Neighborhood column using `isnull`.

In [5]:
df['Neighborhood'].isnull().sum(axis = 0)

0

To make data frame as asked in the question, the character between neighborhood `/` with `,`.

In [6]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


There are no NaN values in the Neighborhood column. It seems the downloaded table from wikipedia have taken care of these things. Now the dimensions of data frame `df` are printing using `shape`.

In [7]:
print('Data shape:',df.shape)

Data shape: (103, 3)


## Part 2

## Location of neighborhood

Here I am using geocoding using ArcGIS developer platform to convert the neighborhood addresses into location coordinates. The coordinates are then stored in `lat_lng` list.

In [8]:
lat_lng = []
for pcode in df['Postal code']:
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(pcode))
        lat_lng_coords = g.latlng
        lat_lng.append(lat_lng_coords)

Converting list of `lat_lng` into a data frame of coordinates with Latitude and Longitude column names.

In [9]:
lat_lng = pd.DataFrame(lat_lng, columns = ['Latitude', 'Longitude'])
lat_lng.head()

,Latitude,Longitude
0,43.752935,-79.335641
1,43.728102,-79.311890
2,43.650964,-79.353041
3,43.723265,-79.451211
4,43.661790,-79.389390


Let's join the data frames `df` and `lat_lng`as `final_df`.

In [10]:
final_df = pd.concat([df, lat_lng], axis=1)
final_df.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
5,M9A,Etobicoke,Islington Avenue,43.667481,-79.528953
6,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
7,M3B,North York,Don Mills,43.748900,-79.357220
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529


## Part 3

## Creating the map of Toronto with neighborhood as marker on it

To simplify the analysis, we are only focusing on the neighborhood in Toronto. Now slice the `final_df` and create a data frame for Toronto

In [11]:
toronto_df = final_df[final_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


Toronto coordinates are 43.6532 and -79.3832. Next, exploring the neighborhood using FourSquare APIs.

In [12]:
# create map of Toronto
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to toronto
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

#### Analysis of 1<sup>st</sup> neighborhood in Toronto

Let start our focus on the 1st neighborhood of Toronto. Store the name, latitude and longitude of 1st neighborhood in new variables.

In [13]:
first_neighborhood = toronto_df.loc[0,'Neighborhood']
first_lat = toronto_df.loc[0,'Latitude']
first_lng = toronto_df.loc[0,'Longitude']
print('1st neighborhood is {} and its coordinates are {} and {}'.format(first_neighborhood,first_lat,first_lng))

1st neighborhood is Regent Park, Harbourfront and its coordinates are 43.65096410900003 and -79.35304116399999


Signing in FourSquare with credential to get API call for exploring neighborhood.

In [14]:
CLIENT_ID = 'Q0CJAXMMYPMPVLVCHLUBBA0FIA3NEHXYSJGDLKNZBPNTCEEX' # your Foursquare ID
CLIENT_SECRET = 'ALNAKLHWU2QHYCB2ASQ2XY5RMDEM5RYR2FWV0NSXIER3MHHJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [15]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format (
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_lat, 
    first_lng, 
    radius, 
    LIMIT)
url
results = requests.get(url).json()

Let's define the get_category_type function!

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Next, I am cleaning the results from json and structured them as data frame.

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Souk Tabule,Mediterranean Restaurant,43.653756,-79.354390
1,Young Centre for the Performing Arts,Performing Arts Venue,43.650825,-79.357593
2,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
3,Cluny Bistro & Boulangerie,French Restaurant,43.650565,-79.357843
4,BATLgrounds,Athletics & Sports,43.647088,-79.351306


In [18]:
print('{} venues were returned by Foursquare for 1st neighborhood.'.format(nearby_venues.shape[0]))
nearby_venues['categories']

22 venues were returned by Foursquare for 1st neighborhood.


0     Mediterranean Restaurant
1        Performing Arts Venue
2               Chocolate Shop
3            French Restaurant
4           Athletics & Sports
5          Distribution Center
6                 Tech Startup
7           Athletics & Sports
8                         Café
9           Mexican Restaurant
10                         Pub
11                      Bakery
12                     Theater
13                         Pub
14                 Coffee Shop
15                        Café
16             Thai Restaurant
17                         Pub
18                        Bank
19          Seafood Restaurant
20                  Food Truck
21                       Trail
Name: categories, dtype: object

## Analysis of all neighborhood in Toronto

Let's repeat the same process for all the neighborhood in Toronto by create a function `getNearbyVenues`.

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town

Let's check the size the toronto_venues

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1617, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
4,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports


Count the number of venues for each neighborhood

In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,65,65,65,65,65,65
"Brockton, Parkdale Village, Exhibition Place",44,44,44,44,44,44
Business reply mail Processing CentrE,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",65,65,65,65,65,65
Central Bay Street,64,64,64,64,64,64
Christie,12,12,12,12,12,12
Church and Wellesley,82,82,82,82,82,82
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,25,25,25,25,25,25


How many unique categories in venue data?

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 221 uniques categories.


### Analyze each neighborhood

In [24]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")
toronto_onehot = pd.concat([toronto_venues['Neighborhood'], toronto_onehot.drop(['Neighborhood'], axis = 1)], axis = 1)
toronto_onehot.head()
print(toronto_onehot.shape)

(1617, 221)


Now I am grouping all the neighborhood and listing the frequency all of the catogory

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head(5)

(38, 221)


,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0,0.000000,0.015385,0.000000,0.000000,0.0,0.015385,0.0,...,0.000000,0.0,0.0,0.0,0.015385,0.000000,0.000000,0.000000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.022727,0.0,0.000000,0.022727,0.022727,0.000000,0.0,0.000000,0.0,...,0.022727,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,Business reply mail Processing CentrE,0.000000,0.0,0.030000,0.010000,0.010000,0.020000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.020000,0.000000,0.000000,0.010000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.000000,0.000000,0.000000,0.015385,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,Central Bay Street,0.000000,0.0,0.015625,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.015625,0.015625,0.015625,0.0,0.0


Print top 5 most common venues for each neighborhood.

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1  Seafood Restaurant  0.05
2          Restaurant  0.05
3                Café  0.03
4            Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.07
2             Pizza Place  0.05
3  Thrift / Vintage Store  0.05
4               Gift Shop  0.05


----Business reply mail Processing CentrE----
                 venue  freq
0          Coffee Shop  0.08
1                Hotel  0.05
2           Restaurant  0.04
3  American Restaurant  0.03
4                 Café  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport----
               venue  freq
0        Coffee Shop  0.08
1               Café  0.06
2         Restaurant  0.06
3  French Restaurant  0.05
4               Park  0.05


----Central Bay Street----
                  venue  freq
0   

Writing function to put the above data into a data frame

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now considering top 10 venues for each neighborhood.

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Restaurant,Beer Bar,Farmers Market,Bakery,Italian Restaurant,Breakfast Spot,Cocktail Bar,Cheese Shop
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Gift Shop,Pizza Place,Thrift / Vintage Store,Accessories Store,Chiropractor,Boutique,Brewery,French Restaurant
2,Business reply mail Processing CentrE,Coffee Shop,Hotel,Restaurant,Café,American Restaurant,Italian Restaurant,Seafood Restaurant,Gym,Movie Theater,Sushi Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Restaurant,Park,French Restaurant,Bar,Italian Restaurant,Lounge,Speakeasy,Gym / Fitness Center
4,Central Bay Street,Coffee Shop,Clothing Store,Plaza,Tea Room,Electronics Store,Gym / Fitness Center,Japanese Restaurant,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop


## Cluster Neighborhoods

Focus is on to cluster the neighborhoods using k-means clustering approach with k =4

In [29]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)

Now to create a data frame for each neighborhoor with top 10 venues

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# Combine the data
toronto_merged = toronto_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,1.0,Pub,Café,Athletics & Sports,Bakery,Theater,Coffee Shop,French Restaurant,Mediterranean Restaurant,Distribution Center,Tech Startup
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,1.0,Coffee Shop,Café,Yoga Studio,Park,Burrito Place,Sandwich Place,College Auditorium,College Cafeteria,Creperie,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,1.0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Café,Hotel,Cosmetics Shop,Italian Restaurant,Diner,Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,1.0,Café,Coffee Shop,Cocktail Bar,Hotel,Cosmetics Shop,American Restaurant,Italian Restaurant,Seafood Restaurant,Restaurant,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,1.0,Health Food Store,Pub,Trail,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [31]:
toronto_merged=toronto_merged.drop(toronto_merged.index[21]).reset_index(drop=True)
toronto_merged=toronto_merged.astype({'Cluster Labels':'int64'})
toronto_merged

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,1,Pub,Café,Athletics & Sports,Bakery,Theater,Coffee Shop,French Restaurant,Mediterranean Restaurant,Distribution Center,Tech Startup
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,1,Coffee Shop,Café,Yoga Studio,Park,Burrito Place,Sandwich Place,College Auditorium,College Cafeteria,Creperie,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Café,Hotel,Cosmetics Shop,Italian Restaurant,Diner,Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,1,Café,Coffee Shop,Cocktail Bar,Hotel,Cosmetics Shop,American Restaurant,Italian Restaurant,Seafood Restaurant,Restaurant,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,1,Health Food Store,Pub,Trail,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
5,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855,1,Coffee Shop,Seafood Restaurant,Restaurant,Beer Bar,Farmers Market,Bakery,Italian Restaurant,Breakfast Spot,Cocktail Bar,Cheese Shop
6,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653,1,Coffee Shop,Clothing Store,Plaza,Tea Room,Electronics Store,Gym / Fitness Center,Japanese Restaurant,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop
7,M6G,Downtown Toronto,Christie,43.668602,-79.420387,0,Café,Grocery Store,Baby Store,Athletics & Sports,Candy Store,Coffee Shop,Playground,Park,Fish Market,Eastern European Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650542,-79.384116,1,Coffee Shop,Café,Restaurant,Gym,Hotel,Thai Restaurant,Bakery,Gym / Fitness Center,Steakhouse,Bar
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.664910,-79.438664,1,Park,Grocery Store,Smoke Shop,Café,Brazilian Restaurant,Furniture / Home Store,Bus Line,Bank,Bakery,Gym


Now visualize the clusters in map!

In [32]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

#### cluster 1

In [33]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('Most importnat venues in 1st cluster are \n{}'.format(cluster1['1st Most Common Venue'].value_counts()))
cluster1

Most importnat venues in 1st cluster are 
Playground    2
Bus Line      1
Café          1
Name: 1st Most Common Venue, dtype: int64


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Café,Grocery Store,Baby Store,Athletics & Sports,Candy Store,Coffee Shop,Playground,Park,Fish Market,Eastern European Restaurant
12,East Toronto,0,Bus Line,Business Service,Park,Grocery Store,Discount Store,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
22,Central Toronto,0,Playground,Gym Pool,Park,Garden,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
32,Downtown Toronto,0,Playground,Park,Grocery Store,Candy Store,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


#### Cluster 2 -Caffee shops

In [34]:
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('Most importnat venues in 2nd cluster are \n{}'.format(cluster['1st Most Common Venue'].value_counts()))
cluster

Most importnat venues in 2nd cluster are 
Coffee Shop             16
Café                     5
Convenience Store        3
Harbor / Marina          1
Pub                      1
Park                     1
Health Food Store        1
Fast Food Restaurant     1
Baseball Field           1
Bar                      1
Dessert Shop             1
Name: 1st Most Common Venue, dtype: int64


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Pub,Café,Athletics & Sports,Bakery,Theater,Coffee Shop,French Restaurant,Mediterranean Restaurant,Distribution Center,Tech Startup
1,Downtown Toronto,1,Coffee Shop,Café,Yoga Studio,Park,Burrito Place,Sandwich Place,College Auditorium,College Cafeteria,Creperie,Diner
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Café,Hotel,Cosmetics Shop,Italian Restaurant,Diner,Restaurant
3,Downtown Toronto,1,Café,Coffee Shop,Cocktail Bar,Hotel,Cosmetics Shop,American Restaurant,Italian Restaurant,Seafood Restaurant,Restaurant,Breakfast Spot
4,East Toronto,1,Health Food Store,Pub,Trail,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
5,Downtown Toronto,1,Coffee Shop,Seafood Restaurant,Restaurant,Beer Bar,Farmers Market,Bakery,Italian Restaurant,Breakfast Spot,Cocktail Bar,Cheese Shop
6,Downtown Toronto,1,Coffee Shop,Clothing Store,Plaza,Tea Room,Electronics Store,Gym / Fitness Center,Japanese Restaurant,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Gym,Hotel,Thai Restaurant,Bakery,Gym / Fitness Center,Steakhouse,Bar
9,West Toronto,1,Park,Grocery Store,Smoke Shop,Café,Brazilian Restaurant,Furniture / Home Store,Bus Line,Bank,Bakery,Gym
10,Downtown Toronto,1,Harbor / Marina,Theme Park,Park,Farm,Yoga Studio,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


#### CLuster 3

In [35]:
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('Most importnat venues in 3rd cluster are \n{}'.format(cluster['1st Most Common Venue'].value_counts()))
cluster

Most importnat venues in 3rd cluster are 
IT Services    1
Name: 1st Most Common Venue, dtype: int64


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,IT Services,Yoga Studio,Dumpling Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 4

In [36]:
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('Most importnat venues in 4th cluster are \n{}'.format(cluster['1st Most Common Venue'].value_counts()))
cluster

Most importnat venues in 4th cluster are 
Bus Line    1
Name: 1st Most Common Venue, dtype: int64


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Bus Line,Swim School,Yoga Studio,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


### **Summary:**  Cluter 2 represnts a big cluster of coffee shops. 